In [1]:
import general_utils
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

In [2]:
from algorithm_utils import *
import general_utils as gu

In [3]:
timeframe = "15m"
higher_timeframe = general_utils.find_higher_timeframe(timeframe)

In [4]:
testing_length = 500
start_index = -8 * testing_length - 1
pair_name = "BTCUSDT"

original_pair_df: pd.DataFrame = gu.load_local_data(pair_name=pair_name, timeframe=timeframe)

if testing_length > 0:
    pair_df: pd.DataFrame = original_pair_df.iloc[start_index:start_index + testing_length].reset_index(drop=True)
else:
    pair_df: pd.DataFrame = original_pair_df

# Higher timeframe data for determining starting point
htf_pair_df: pd.DataFrame = gu.load_higher_tf_data(pair_name=pair_name, timeframe=higher_timeframe)

print("LTF length is", len(pair_df))
print("HTF length is", len(htf_pair_df))

LTF length is 500
HTF length is 5845


In [5]:
gu.reset_logs()

print("Original pair_df starting point is at time", pair_df.iloc[0].time)

algo = Algo(pair_df, pair_name, allowed_verbosity=3)
initial_data_start_time = algo.pair_df.iloc[0].time

corrected_pair_df = create_filtered_pair_df_with_corrected_starting_point(htf_pair_df, initial_data_start_time,
                                                                          original_pair_df, timeframe, higher_timeframe)

print("Updated pair_df starting point to", corrected_pair_df.iloc[0].time)

algo = Algo(corrected_pair_df, "BTCUSDT", allowed_verbosity=3)

algo.init_zigzag(last_pivot_type="valley", last_pivot_candle_pdi=0)
h_o_starting_point: int = algo.zigzag_df.iloc[0].pdi

algo.calc_h_o_zigzag(h_o_starting_point)

# corrected_pair_df = pair_df.copy()

# print("Updated pair_df starting point to", corrected_pair_df.iloc[0].time)

# algo = Algo(corrected_pair_df, pair_name, allowed_verbosity=3)

# algo.init_zigzag(last_pivot_type="peak", last_pivot_candle_pdi=3)

# h_o_starting_point: int = algo.zigzag_df.iloc[0].pdi

# algo.calc_h_o_zigzag(h_o_starting_point)


# algo.h_o_indices.append(87)
# algo.h_o_indices.append(315)
# algo.h_o_indices.append(431)
# print(algo.find_lpls())
# h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
# pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
# bos_df = find_confirmed_boss(pbos_df, pair_df)
# lpl_df = find_lpls(bos_df, zigzag_df)
# lplb_df = find_lplbs(bos_df, zigzag_df)

# boxes_list = []
# for lpl in lpl_df.itertuples():
#     start_index = lpl.pair_df_index
#     base_candle = Candle.create(pair_df.iloc[lpl.pair_df_index])
#     box_type = "long" if lpl.pivot_type == "valley" else "short"
#     box_to_add = Box(base_candle, b|ox_type)
#     box_to_add.check_box_ent

#     ries(pair_df)
#     boxes_list.append(box_to_add)


Original pair_df starting point is at time 2024-07-21 08:00:00
Updated pair_df starting point to 2024-07-13 22:15:00
('Added starting point', np.int64(0))
('',)
('Trend type is', 'ascending')
('Changing breaking_pdi to', np.int64(18))
('Changing breaking_pdi to', np.int64(34))
('Starting pattern at', np.int64(0))
('Broken LPL is at', np.int64(34))
('Added BOS', 40)
('HO indices', [np.int64(0), 40])
('CHOCH threshold is at', np.int64(0), 'BOS threshold is at', 40)
('Candle at index', np.int64(92), 'broke the last PBOS #', 40, 'with its close price')
('BOS #', 40, 'break at', np.int64(92))
('Added extremum of type', 'lowest low', 'at', np.int64(65))
('Setting pattern start to', np.int64(65))
('',)
('Trend type is', 'ascending')
('Changing breaking_pdi to', np.int64(103))
('Changing breaking_pdi to', np.int64(110))
('Changing breaking_pdi to', np.int64(121))
('Changing breaking_pdi to', np.int64(132))
('Starting pattern at', np.int64(65))
('Broken LPL is at', np.int64(132))
('Added BOS', 

In [21]:
pt = gu.PlottingTool(width=1400, height=800)
pt.draw_candlesticks(corrected_pair_df)
pt.draw_zigzag(algo.zigzag_df)

pt.draw_points_with_label(algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pdi,
                          algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pivot_value,
                          draw_line=True)

for segment in [algo.segments[-5]]:
    segment.filter_candlestick_range(algo)
    segment.find_order_blocks(algo)
    pt.draw_segment_bbox(segment)
    for ob in segment.ob_list:
        if len(ob.price_reentry_indices) == 0:
            continue
            
        if segment.ob_formation_start_pdi < ob.price_reentry_indices[0] < segment.end_pdi:
            pt.draw_box(ob, algo.pair_df.iloc[-1].name)
            ob.enter(ob.price_reentry_indices[0])
            print(ob)
            print("ENTERED AT", ob.price_reentry_indices[0])
    break
    
pt.show()

OB 1609/2024.7.30/16:30:00S (short)
ENTERED AT 1630
